In [1]:
import os, sys
sys.path.append(os.path.dirname("../"))

In [2]:
from utils.results import Results
from utils.db_connector import DBConnector
import pandas as pd


In [21]:
db = DBConnector()
projects = db.run_query("""SELECT "ProjectID","Name" FROM "Projects" WHERE "Name" LIKE '%FINAL%';""")

In [22]:
projects = pd.DataFrame(projects, columns=["ProjectID", "Name"])

In [23]:
projects.head()

,ProjectID,Name
0,f71e625e-4518-4bc1-8dbe-d5af3f723e07,Std-U1-Cn-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL
1,0c61fb04-3852-4be7-a591-44a09e28e03b,Std-U2-Cn-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL
2,22c37276-b207-4b5f-a971-07ee81e32163,Std-U3-Cn-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL
3,7e66cc56-0421-4edb-b4bb-f22a3bcf755c,Std-U0-Cy-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL
4,4adf729d-353c-43b9-b25e-143afc605f5d,Std-U1-Cy-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL


In [ ]:
for index, row in projects.iterrows():
    project_id = row["ProjectID"]
    project_name = row["Name"]
    print(f"Processing {project_name} ({project_id})")
    
    # Get the results for the project
    try:
        results = Results(project_id)
        metrics = {
            "MCC": results.get_mcc(),
            "bacc": results.get_balanced_accuracy(),
            "recall": results.get_recall(),
            "precision": results.get_precision(),
            "spec": results.get_specificity(),
            "npv": results.get_npv(),
            "acc": results.get_accuracy()
        }
        for metric, value in metrics.items():
            projects.at[index, metric] = value
    except Exception as e:
        print(f"Error processing {project_name} ({project_id}): {e}")
        projects.at[index, "MCC"] = None
        projects.at[index, "bacc"] = None
        projects.at[index, "recall"] = None
        projects.at[index, "precision"] = None
        projects.at[index, "spec"] = None
        projects.at[index, "npv"] = None
        projects.at[index, "acc"] = None
        

Processing Std-U1-Cn-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL (f71e625e-4518-4bc1-8dbe-d5af3f723e07)
Processing Std-U2-Cn-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL (0c61fb04-3852-4be7-a591-44a09e28e03b)
Processing Std-U3-Cn-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL (22c37276-b207-4b5f-a971-07ee81e32163)
Processing Std-U0-Cy-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL (7e66cc56-0421-4edb-b4bb-f22a3bcf755c)
Processing Std-U1-Cy-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL (4adf729d-353c-43b9-b25e-143afc605f5d)
Processing Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL (f7bc5d5f-c6cb-4949-a923-2aa32a156710)
Processing Std-U3-Cy-Rn-EXn-INn-A-SH0-EXPn-RL4SE-FINAL (bff14c8b-5f9b-4857-bf60-236626978566)
Processing Std-U1-Cn-Rn-EXn-INn-A-SH0-EXPn-LC-FINAL (38360a5c-ba2a-47b7-ad6d-a25067fc7f58)
Processing Std-U2-Cn-Rn-EXn-INn-A-SH0-EXPn-LC-FINAL (c4178e0f-5dd5-4b56-9e25-0c6af5516f25)
Processing Std-U3-Cn-Rn-EXn-INn-A-SH0-EXPn-LC-FINAL (d594e24b-fdfa-4504-8e8f-40f2e10b7ab0)
Processing Std-U0-Cy-Rn-EXn-INn-A-SH0-EXPn-LC-FINAL (e5a0304c-7b16-41

In [26]:
print(projects.shape)
print(projects.dropna().shape)

(101, 9)
(99, 9)


In [27]:
projects.dropna(inplace=True)

In [28]:
projects.to_excel("chp5_analysis.xlsx", index=False)

In [ ]:
projects.Name[0].split("-")[]

['Std', 'U1', 'Cn', 'Rn', 'EXn', 'INn', 'A', 'SH0', 'EXPn', 'RL4SE', 'FINAL']

In [51]:
db = DBConnector()
projects = db.run_query("""
SELECT
  "ProjectID","Name"
FROM
  kumargau."Projects"
WHERE
  "Name" NOT LIKE '%BIBTEX%'
  AND "Name" LIKE '%U2%%FINAL%%GPT%';
""")
projects = pd.DataFrame(projects, columns=["ProjectID", "Name"])
projects = pd.concat([projects,projects.Name.str.split("-", expand=True)], axis=1)

In [52]:
projects.columns
columns_required = {0: 'S', 1:'U', 2: 'C', 3: 'R', 4: 'EX', 5: 'IN', 6: 'A', 7:'SH', 8:'EXP', 9:'RQ', 10:'Type', 11:'DATASET'}
projects = projects.rename(columns=columns_required)
projects = projects.drop(columns=[12, 13, 14, 15, 16])

In [53]:
projects.S = projects.S.apply(lambda x: "T" if x=="St" else "D")
projects.U = projects.U.apply(lambda x: x.split("U")[-1])
projects.C = projects.C.apply(lambda x: "conf" if x=="Cy" else "-")
projects.R = projects.R.apply(lambda x: "reason" if x=="Ry" else "-")
projects.EX = projects.EX.apply(lambda x: "ex" if x=="EXy" else "-")
projects.IN = projects.IN.apply(lambda x: "in" if x=="INy" else "-")
projects.A = projects.A.apply(lambda x: "-")
projects.SH = projects.SH.apply(lambda x: x.split("SH")[-1])
projects.EXP = projects.EXP.apply(lambda x: "exp" if x=="EXPy" else "no")
projects.RQ = projects.RQ.apply(lambda x: "RQ" if x=="RQy" else "-")


In [56]:
projects.head()
original_df = projects.copy()

In [57]:
for index, row in projects.iterrows():
    project_id = row["ProjectID"]
    project_name = row["Name"]
    print(f"Processing {project_name} ({project_id})")
    
    # Get the results for the project
    try:
        results = Results(project_id)
        metrics = results.get_results()
        for metric, value in metrics.items():
            projects.at[index, metric] = value
    except Exception as e:
        print(f"Error processing {project_name} ({project_id}): {e}")

Processing Std-U2-Cy-Rn-EXn-INn-A-SH1-EXPn-RQn-SIMPLE-LC-FINAL-GPT-4-TURBO (e6ee2310-a9b6-4555-bb3b-f63c775f69de)
Processing Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQn-SIMPLE-UPDATECOLLABMDE-FINAL-GPT-4-TURBO (39c1cc76-a57b-4c76-86ed-97ccab1f1701)
Processing Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQn-SIMPLE-MPM4CPS-FINAL-GPT-4-TURBO (8ff0205d-ab97-4696-9b49-85cf96756053)
Processing Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQy-SIMPLE-LC-FINAL-GPT-4-TURBO (09d94d7a-2259-499c-8223-a74da4fdeed5)
Processing Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQy-SIMPLE-MPM4CPS-FINAL-GPT-4-TURBO (f9152bd1-bc8a-4127-af0c-2ffbc65e01ba)
Processing Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQy-SIMPLE-UPDATECOLLABMDE-FINAL-GPT-4-TURBO (d5973d35-70d3-42df-b6b7-c53848d29daf)
Processing Std-U2-Cy-Rn-EXn-INn-A-SH1-EXPn-RQy-SIMPLE-LC-FINAL-GPT-4-TURBO (7970d4de-2a92-4d16-949e-5902e09ccfef)
Processing Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQn-SIMPLE-LC-FINAL-GPT-4-TURBO (c94c3f82-669c-4aa6-a576-d125a6fea44f)
Processing Std-U2-Cy-Rn-EXn-INn-A-SH1-EXPn-RQy-SIMPL

In [58]:
projects.head()

,ProjectID,Name,S,U,C,R,EX,IN,A,SH,...,specificity,mcc,balanced_accuracy,miss_rate,f2_score,wss,wss@95,npv,g_mean,general_performance_score
0,e6ee2310-a9b6-4555-bb3b-f63c775f69de,Std-U2-Cy-Rn-EXn-INn-A-SH1-EXPn-RQn-SIMPLE-LC-...,D,2,conf,-,-,-,-,1,...,0.6819,0.6745,0.8302,0.0216,0.5296,0.6144,0.5860,0.9976,0.8168,0.8037
1,39c1cc76-a57b-4c76-86ed-97ccab1f1701,Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQn-SIMPLE-UPD...,D,2,conf,-,-,-,-,0,...,0.8692,0.6989,0.8030,0.2632,0.5570,0.5666,0.7797,0.9793,0.8003,0.7976
2,8ff0205d-ab97-4696-9b49-85cf96756053,Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQn-SIMPLE-MPM...,D,2,conf,-,-,-,-,0,...,0.7245,0.7079,0.7080,0.3084,0.6994,0.1989,0.4573,0.6827,0.7078,0.7077
3,09d94d7a-2259-499c-8223-a74da4fdeed5,Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQy-SIMPLE-LC-...,D,2,conf,-,-,-,-,0,...,0.8644,0.7651,0.9151,0.0342,0.7071,0.7739,0.7581,0.9971,0.9137,0.9123
4,f9152bd1-bc8a-4127-af0c-2ffbc65e01ba,Std-U2-Cy-Rn-EXn-INn-A-SH0-EXPn-RQy-SIMPLE-MPM...,D,2,conf,-,-,-,-,0,...,0.7895,0.8119,0.8114,0.1667,0.8278,0.3034,0.4201,0.8182,0.8111,0.8108


In [59]:
projects.to_clipboard(index=False)